# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('./sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
#with open('token_key.csv', 'w') as f:
    #pass
#IEX_CLOUD_API_TOKEN = ''    
with open('token_key.csv', 'r') as f:
    IEX_CLOUD_API_TOKEN = f.read()
    
#IEX_CLOUD_API_TOKEN  

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SRK GAE MNTE(BGQLA DTLA/SSLOCNAE)', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'fialfioc', 'close': None, 'closeTime': None, 'closeSource': 'lofcaifi', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 130.6, 'latestSource': 'Close', 'latestTime': 'January 12, 2021', 'latestUpdate': 1683806490903, 'latestVolume': None, 'iexRealtimePrice': 131.41, 'iexRealtimeSize': 924, 'iexLastUpdated': 1683352232083, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 130.65, 'previousVolume': 102346170, 'change': -0.19, 'changePercent': -0.0014, 'volume': None, 'iexMarketPercent': 21.40343053436009, 'iexVolume': 848955, 'avgTotalVolume': 119682797, 'iexBidPrice':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [8]:
price = data['latestPrice']
market_cap = data['marketCap'] / 1000000000000
market_cap

2.236457469951

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares To Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares To Buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
df = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    pd_series = pd.Series([
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
     index = my_columns,
     #ignore_index=True
    )
    df = df.append(pd_series,  ignore_index=True)

In [12]:
df.head(20)

,Ticker,Stock Price,Market Capitalization,Number of Shares To Buy
0,A,127.79,39669052032,N/A
1,AAL,15.51,9444130471,N/A
2,AAP,183.32,12554547546,N/A
3,AAPL,131.40,2207466138511,N/A
4,ABBV,114.11,201993695562,N/A
5,ABC,111.79,21847582576,N/A
6,ABMD,339.59,15288642838,N/A
7,ABT,112.81,202411985261,N/A
8,ACN,265.76,173882465935,N/A
9,ADBE,477.98,235028398796,N/A


In [13]:
# checking how to group in tens
for a in range(0,100+10,10):
    print(a)

0
10
20
30
40
50
60
70
80
90
100


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [14]:
# code from stackoverflow
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [15]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    # take each subset as a string and append to the list
    symbol_strings.append(','.join(symbol_groups[i]))

df_batch = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    # url copied from the iex cloud api documentation
    #common_api_url = 'https://sandbox.iexapis.com/stable'
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    
    #split the symbol strings  
    for symbol in symbol_string.split(','):
        #print(symbol)
        #make a pd Series
        pd_series = pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ], index = my_columns)
        # append to the dataframe
        df_batch = df_batch.append(pd_series, ignore_index = True)
        
df_batch.head(20)

,Ticker,Stock Price,Market Capitalization,Number of Shares To Buy
0,A,132.46,40407010609,N/A
1,AAL,16.03,9364188134,N/A
2,AAP,178.73,12275028485,N/A
3,AAPL,134.70,2210400666632,N/A
4,ABBV,112.19,199494431067,N/A
5,ABC,107.02,21974484833,N/A
6,ABMD,333.74,14789676768,N/A
7,ABT,108.94,198717533265,N/A
8,ACN,267.58,172467892677,N/A
9,ADBE,479.02,226457270623,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.
(This is because it is better done base on the portfolio of the individual or the company.)

We'll do that next.

In [43]:

# use try and except to validate the input given by the user
try:
    portfolio_size = input('Enter the value of your portfolio: ')
    # force the input to be changed to float
    val = float(portfolio_size)
    #print(val)
except ValueError:
    print("That's not a number \n Please try again!")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)
    

    

Enter the value of your portfolio: 10000000


In [56]:
position_size = val / len(df_batch)
number_of_apple_shares = position_size / 500    # apple share cost 500
# use the number of share of each symbols to fill of the df table
for i in range(0, len(df)):
    df.loc[i,'Number of Shares To Buy'] = math.floor(position_size / df.loc[i,'Stock Price'])
    
df.head(20)
#number_of_apple_shares
#math.floor(number_of_apple_shares)     # to round down

,Ticker,Stock Price,Market Capitalization,Number of Shares To Buy
0,A,127.79,39669052032,154
1,AAL,15.51,9444130471,1276
2,AAP,183.32,12554547546,108
3,AAPL,131.40,2207466138511,150
4,ABBV,114.11,201993695562,173
5,ABC,111.79,21847582576,177
6,ABMD,339.59,15288642838,58
7,ABT,112.81,202411985261,175
8,ACN,265.76,173882465935,74
9,ADBE,477.98,235028398796,41


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [87]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'Recommended Trade', index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [88]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        'bg_color': background_color,
        'font_color': font_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    { 
        'num_format': '$0.00',
        'bg_color': background_color,
        'font_color': font_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'bg_color': background_color,
        'font_color': font_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [89]:
# writer.sheets['Recommended Trade'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trade'].set_column('D:D', 18, string_format)
# writer.save()

#to format the headers
#writer.sheets['Recommended Trade'].write('A1', 'Ticker', string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [90]:
columns = {
    'A' : ['Ticker',string_format],
    'B' : ['Stock Price', dollar_format],
    'C' : ['Market Capitalization',dollar_format],
    'D' : ['Number of Shares To buy',integer_format]
}

# for body 
for column in columns.keys():
    writer.sheets['Recommended Trade'].set_column(f'{column}:{column}', 18, columns[column][1])
    
# for headers

for keys in columns.keys():
    writer.sheets['Recommended Trade'].write(f'{keys}'+'1',columns[keys][0], columns[keys][1])

# Note u can decide to use oone loop for bothbody and the header

## Saving Our Excel Output

Saving our Excel file is very easy:

In [91]:
writer.save()